# Дисклеймер
Эту тетрадку нужно запускать в колабе или на машине с gpu.

In [ ]:
# !apt-get install unzip

In [31]:
import os
os.environ["KERAS_BACKEND"] = "torch"

import keras
import torch
print(keras.__version__)

3.1.1


In [2]:
# %pip install tokenizers matplotlib scikit-learn

Большая часть этого семинара пересекается с видео от Andrey Karpathy - https://www.youtube.com/watch?v=kCc8FmEb1nY Возможно его объяснения покажутся вам лучше. 
Его видео более детальное и объемное, но фокус в нем скорее на GPT (декодере) для простой генерации текста, и еще оно на torch. 

# Транформеры для машинного перевода

На huggingface очень много предобученных трансформеров, которые позволят вам решить очень большой процент рабочих/исследовательских задач. Однако бывают ситуации, когда нужной предобученной модели нет или она работает не очень хорошо. Обучить очень большую модель вряд ли получится (нужны видеокарты), но вот для средних моделей и специфичных задач может хватить даже колаба/каггла.

Давайте обучим свой небольшой трансформер на задаче машинного перевода. Это самая классическая seq2seq задача и подход, который мы размерем, применим и для других аналогичных задач. Для того, чтобы обучить модель решать какую-то другую seq2seq задачу вам нужно будет только подставить другие данные. Под seq2seq вообще при желании можно подвести любую задачу (классификация - это предсказание последовательности длинной 1 или можно предсказыть названия классов текстом). 

In [3]:
# import tensorflow as tf
import keras
from tokenizers import BertWordPieceTokenizer

from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import normalizers
from tokenizers.normalizers import Lowercase
from tokenizers.trainers import WordPieceTrainer
from tokenizers import decoders

import os
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from string import punctuation
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

Данные взяты вот отсюда - https://opus.nlpl.eu/opus-100.php (раздел с отдельными языковыми парами)

In [337]:
# !wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
# !wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.en

--2024-03-20 21:26:41--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.ru
Resolving data.statmt.org (data.statmt.org)... 129.215.32.28
Connecting to data.statmt.org (data.statmt.org)|129.215.32.28|:443... 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


connected.
HTTP request sent, awaiting response... 200 OK
Length: 305669 (299K)
Saving to: ‘opus.en-ru-test.ru’

opus.en-ru-test.ru  100%[===================>] 298.50K   345KB/s    in 0.9s    

2024-03-20 21:26:43 (345 KB/s) - ‘opus.en-ru-test.ru’ saved [305669/305669]

--2024-03-20 21:26:43--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.en
Resolving data.statmt.org (data.statmt.org)... 129.215.32.28
Connecting to data.statmt.org (data.statmt.org)|129.215.32.28|:443... 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


connected.
HTTP request sent, awaiting response... 200 OK
Length: 173307 (169K)
Saving to: ‘opus.en-ru-test.en’

opus.en-ru-test.en  100%[===================>] 169.25K   245KB/s    in 0.7s    

2024-03-20 21:26:44 (245 KB/s) - ‘opus.en-ru-test.en’ saved [173307/173307]



In [5]:
# в русскоязычных данных есть \xa0 вместо пробелов, он может некорректно обрабатываться токенизатором
text = open('opus.en-ru-train.ru').read().replace('\xa0', ' ')
f = open('opus.en-ru-train.ru', 'w')
f.write(text)
f.close()

In [412]:
en_sents = open('opus.en-ru-train.en').read().lower().splitlines()
ru_sents = open('opus.en-ru-train.ru').read().lower().splitlines()

Пример перевода с английского на русский

In [7]:
en_sents[-1], ru_sents[-1]

('so what are you thinking?', 'ну и что ты думаешь?')

Как обычно нам нужен токенизатор, а точнее даже 2, т.к. у нас два корпуса. Будем использовать wordpiece токенайзер из библиотеки tokenizers от huggingface. Wordpiece разбивает текст на токены, которые могут быть как целыми словами, так и кусками слов и даже отдельными символами. 
В токенизатор мы передаем еще несколько специальных символов - UNK (это символ замена для неизвестных слов), START - этот токен будет добавлен в начало текста, END - этот символ будет добавлен в конце, PAD - это индекс паддинга

В токенезаторе для целевого языка по идее нам не нужен UNK, потому что в переводном тексте технических токенов быть не должно, но мы не можем без него обойтись, потому что мы ограничиваем размер словаря. Можно было бы построить словарь так, чтобы в нем были все возможные токены и тогда UNK был бы не нужен, но тогда размер словаря будет слишком большим. Но мы всегда можем отфильтровать UNK из перевода, просто не разрешая модели предсказывать UNK.

In [8]:
tokenizer_en = Tokenizer(WordPiece(), )
tokenizer_en.normalizer = normalizers.Sequence([Lowercase()])
tokenizer_en.pre_tokenizer = Whitespace()

trainer_en = WordPieceTrainer(
          vocab_size=30000, special_tokens=["[UNK]", "[PAD]"])
tokenizer_en.train(files=["opus.en-ru-train.en"], trainer=trainer_en )

tokenizer_ru = Tokenizer(WordPiece(), )
tokenizer_ru.normalizer = normalizers.Sequence([Lowercase()])
tokenizer_ru.pre_tokenizer = Whitespace()

trainer_ru = WordPieceTrainer(
          vocab_size=30000, special_tokens=["[UNK]", "[PAD]", "[START]", "[END]", ])
tokenizer_ru.train(files=["opus.en-ru-train.ru"], trainer=trainer_ru )

In [9]:
tokenizer_en.decoder = decoders.WordPiece()
tokenizer_ru.decoder = decoders.WordPiece()

Когда мы будет раскодировать предсказанные индексы в слова нам пригодится декодер. Лучше использовать готовый из той же библиотеки. Он сам склеит символьные нграммы в слова. Без него нам нужно было бы писать правила, чтобы избавиться от вот такого

In [10]:
# без декодера
' '.join(tokenizer_ru.encode('Пример текста с редким словом').tokens)

'пример текста с ред ##ким словом'

In [11]:
# c готовым декодером
tokenizer_ru.decoder.decode(tokenizer_ru.encode('Пример текста с редким словом').tokens)

'пример текста с редким словом'

### ВАЖНО!

Токенизатор - это неотъемлимая часть модели, поэтому не забывайте сохранять токенизатор вместе с моделью. Если вы забудете про это и переобучите токенизатор, то индексы токенов разойдутся и веса модели будут бесполезны. 

In [13]:
# раскоментируйте эту ячейку при обучении токенизатора
# а потом снова закоментируйте чтобы при перезапуске не перезаписать токенизаторы
# tokenizer_en.save('tokenizer_en')
# tokenizer_ru.save('tokenizer_ru')

In [14]:
tokenizer_en = Tokenizer.from_file("tokenizer_en")
tokenizer_ru = Tokenizer.from_file("tokenizer_ru")

##### Переводим текст в индексы. 

Для исходного языка нам не нужно никаких дополнительных токенов, поэтому просто вызываем токенизатор.   
А вот для целевого в начало нужно добавить токен '[START]', а в конец '[END]'.  
Эти теги понадобятся, когда мы будем генерировать тексты с уже обученной моделью - тэг [START] мы будем подавать в самом начале перевода, чтобы состояние декодера не было пустым (ниже вы увидите, почему оно не может быть пустым), а [END] будем использовать, чтобы понимать когда остановить перевод (если модель предсказала [END] - значит перевод окончен)

In [413]:
def encode(text, tokenizer, target=False):
    if target:
        return [tokenizer.token_to_id('[START]')] + tokenizer.encode(text).ids + \
                [tokenizer.token_to_id('[END]')]
    else:
        return tokenizer.encode(text).ids 


Кодируем и паддим

In [584]:
X_en = [encode(t, tokenizer_en) for t in en_sents]
X_ru = [encode(t, tokenizer_ru, True) for t in ru_sents]

Как обычно мы должны выбрать максимальную допустимую длину. Максимальные значения слишком большие

In [415]:
max_len_en = np.max([len(x) for x in X_en])
max_len_ru = np.max([len(x) for x in X_ru])

In [416]:
max_len_en, max_len_ru

(17863, 19389)

Если посмотреть на перцентили, то будет видно, что это скорее всего выбросы. Большая часть текстов короткие.

In [417]:
print('Исходный язык')
print('50 % текстов <= ', np.percentile([len(x) for x in X_en], 50))
print('75 % текстов <= ', np.percentile([len(x) for x in X_en], 75))
print('90 % текстов <= ', np.percentile([len(x) for x in X_en], 90))
print('95 % текстов <= ', np.percentile([len(x) for x in X_en], 95))
print('99 % текстов <= ', np.percentile([len(x) for x in X_en], 99))

Исходный язык
50 % текстов <=  10.0
75 % текстов <=  18.0
90 % текстов <=  33.0
95 % текстов <=  45.0
99 % текстов <=  75.0


In [418]:
print('Целевой язык')
print('50 % текстов <= ', np.percentile([len(x) for x in X_ru], 50))
print('75 % текстов <= ', np.percentile([len(x) for x in X_ru], 75))
print('90 % текстов <= ', np.percentile([len(x) for x in X_ru], 90))
print('95 % текстов <= ', np.percentile([len(x) for x in X_ru], 95))
print('99 % текстов <= ', np.percentile([len(x) for x in X_ru], 99))

Целевой язык
50 % текстов <=  11.0
75 % текстов <=  20.0
90 % текстов <=  36.0
95 % текстов <=  48.0
99 % текстов <=  79.0


Давайте для каждого из языков возьмем значения, которые покрывают 95 % всех данных

In [419]:
# обратите внимание, что в seq2seq длины могут быть разными
max_len_en, max_len_ru = 45, 48

По умолчанию паддинг делается нулями, но нам это не подходит, так как в постороенных словарях 0 - это тэг UNK. Вытащим тэг паддинга из словаря и будет использовать его

In [420]:
PAD_IDX = tokenizer_ru.token_to_id('[PAD]')
PAD_IDX

1

In [585]:
X_en = keras.preprocessing.sequence.pad_sequences(
              X_en, maxlen=max_len_en, padding='post', value=PAD_IDX)

X_ru_out = keras.preprocessing.sequence.pad_sequences(
              [x[1:] for x in X_ru], maxlen=max_len_ru-1, padding='post', 
              value=PAD_IDX)

X_ru_dec = keras.preprocessing.sequence.pad_sequences(
              [x[:-1] for x in X_ru], maxlen=max_len_ru-1, 
              padding='post', value=PAD_IDX)

На вход модели мы будем подавать три матрицы.

Первая - X_en - это индексы токенов в английских текстах (тут все как и в предыдущих семинарах)

In [586]:
en_sents[0]

"yeah, that's not exactly..."

In [587]:
X_en[0]

array([3280,   13, 2763,    8,   58, 2808, 5148, 2856,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1], dtype=int32)

А вот русскоязычные тексты мы будем подавать через две матрицы - одна будет использоваться как вход для декодера (X_ru_dec), а вторая как правильные ответы (X_ru_out). 

Если присмотреться, то можно заметить что X_ru_dec и X_ru_out отличаются на 1 шаг (X_ru_out это X_ru_dec сдвинутый вправо)

In [588]:
ru_sents[0]

'да, но не совсем...'

In [589]:
X_ru_dec[0]

array([   2, 2588,   15, 2589, 2513, 5362, 2643,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1], dtype=int32)

In [590]:
X_ru_out[0]

array([2588,   15, 2589, 2513, 5362, 2643,    3,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1], dtype=int32)

Такая структура внутри модели будет использована, чтобы предсказывать следующее слово по текущему контексту. Грубо говоря, модель будет интерпретировать эти две последовательности индексов, как пары контекст-продолжение. 

Вот это:  
2, 2588, 15, 2589, 2513, 5362, 2643   
2588, 15, 2589, 2513, 5362, 2643,    3

Превратится для модели в:  
2 - 2588  
2, 2588 - 15  
2, 2588, 15, 2589 - 2513  
2, 2588, 15, 2589, 2513 - 5362  
2, 2588, 15, 2589, 2513, 5362 - 2643  
2, 2588, 15, 2589, 2513, 5362, 2643 - 3  

(как это будет сделано показано дальше в разделе про маски)

In [496]:
# миллион примеров 
X_en.shape, X_ru_out.shape

((1000000, 45), (1000000, 47))

Разделяем на трейн и тест

In [591]:
(X_en_train, X_en_valid, 
X_ru_dec_train, X_ru_dec_valid, 
X_ru_out_train, X_ru_out_valid) = train_test_split(X_en, 
                                                  X_ru_dec, 
                                                  X_ru_out, 
                                                  test_size=0.05)

Дальше код модели, он взят вот отсюда (с небольшими изменениями) - https://www.tensorflow.org/text/tutorials/transformer

## Имплементация трансформера

Главное в трансформере это механизм внимания. Функция scaled_dot_product_attention преобразует query, key, value вектора в обновленные вектора с помощью механизма внимания.

Визуализация этих шагов (источник: https://jalammar.github.io/illustrated-transformer/):
![](https://jalammar.github.io/images/t/self-attention-output.png "")

Пройдитесь по каждой строчке в этой функции и сопоставьте с визуализацией.

In [429]:
def scaled_dot_product_attention(query, key, value, mask):
    # Считаем скалярное произведение между запросом (query) и ключом (key), транспонируя ключ
    matmul_qk = keras.ops.matmul(query, keras.ops.transpose(key, axes=[0, 1, 3, 2]))

    # Получаем глубину (размерность) ключа и преобразуем ее во float
    depth = keras.ops.cast(key.shape[-1], torch.float32)

    # Делим результат скалярного произведения на квадратный корень из глубины
    # Это делается для уменьшения влияния больших значений и стабилизации градиентов во время обучения
    logits = matmul_qk / keras.ops.sqrt(depth)

    # Если есть маска, применяем ее к логитам, чтобы обнулить нежелательные значения
    if mask is not None:
        logits += (mask * -1e9)

    # Применяем функцию softmax для получения весов внимания
    attention_weights = keras.ops.softmax(logits, axis=-1)

    # Умножаем веса внимания на значения (value) для получения итогового результата
    output = keras.ops.matmul(attention_weights, value)

    return output

В этой функции query, key и value вектора подаются на вход, но как они получаются из эмбедингов?

Это реализовано в классе MultiHeadAttention, который уже является самостоятельным слоем, на котором потом будет построен блок трансформера как в энкодере так и в декодере.

В этом слое можно увидеть три отдельных полносвязных слоя одинаковой размерности (d_model), которые применяются к inputs, чтобы получить q, k, v вектора.  
Можно заметить, что в функции call, разделение на q,k,v происходит еще раньше - они просто достаются из inputs по ключу, как из словаря. Но если немного промотать вперед до encoder_layer , то можно убедиться, что при вызове MultiheadAttention в эти переменные подставляются одно и то же (нажмите CTRL+F и введите #call_mha, чтобы перейти к этому моменту и потом вернутся сюда). 

А если сделать еще один шаг до encoder, то можно увидеть, что такое inputs - это эмбединги слов (CTRL+F и введите #inputs). 



In [430]:
class MultiHeadAttention(keras.layers.Layer):

    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads  # количество голов для внимания
        self.d_model = d_model  # размерность вектора модели

        # Убеждаемся, что размерность модели делится нацело на количество голов
        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads  # размерность каждой головы

        # Создаем полносвязные слои для запроса, ключа и значения
        self.query_dense = keras.layers.Dense(units=d_model)
        self.key_dense = keras.layers.Dense(units=d_model)
        self.value_dense = keras.layers.Dense(units=d_model)

        # Создаем последний полносвязный слой
        self.dense = keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        # Разделяем входные данные на головы
        inputs = keras.ops.reshape(
            inputs, newshape=(batch_size, -1, self.num_heads, self.depth))
        return keras.ops.transpose(inputs, axes=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[
            'value'], inputs.get('mask', None)
        batch_size = query.shape[0]

        # Пропускаем запрос, ключ и значение через соответствующие полносвязные слои
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # Разделяем запрос, ключ и значение на головы 
        # то есть просто разрезаем вектора на num_heads частей 
        # и сравниваем все части между собой
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # Выполняем механизм внимания с масштабированным скалярным произведением
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = keras.ops.transpose(scaled_attention, axes=[0, 2, 1, 3])

        # Объединяем головы вместе (склеиваем векторы в один)
        concat_attention = keras.ops.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))

        # Пропускаем объединенное внимание через дополнительный полносвязный слой
        # Он просто добавляет сложности нашей модели
        outputs = self.dense(concat_attention)

        return outputs


#### Теперь наверное самый сложный момент - почему это MultiHead, а не просто Attention layer? 

В MultiheadAttention используются не по одному q,k,v вектору, а сразу по N векторов, но меньшего размера. В классе выше N задается через парметр num_heads и размерность отдельного q,k или v вектора будет равна d_model/num_heads. 

Внимание рассчитывается между соответствующими парами q и v внутри каждой головы, а итоговый вектор получается объединением результата внимания от всех голов (его размерность будет равна d_model).  

В статье illustrated transformer этот момент визуализируется вот так:
![](https://jalammar.github.io/images/t/transformer_multi-headed_self-attention-recap.png)

Но реализация в TF немного отличается. На этой картинке на шаге 3 показано, что полносвязные слои, через которые получаются q,k,v векторы у каждой головы свои - и их количество не 3, а 3 умножить на количество голов. А вот в TF полносвязных слоя всего 3, но дальше вектора просто разрезаются на куски и начаная с шага 4, все уже совпадает. 

Если задуматься, то разницы между этими подходами нет. Полносвязный слой это матрица, на которую умножается входной вектор, в результате чего получается другой вектор. И каждое значение в новом векторе - это результат попарного умножения и суммы значений в исходном векторе и отдельной колонки в матрице. Таким образом, уможить на матрицу и разрезать результат на три куска - это то же самое, что разрезать матрицу на три куска и умножить вектор на каждый из кусков отдельно:

![](https://i.ibb.co/9Yg580J/image.png)


Давайте еще отдельно посмотрим как происходит разбиение на головы и как потом выглядит скалярное произведение для отдельного куска.

Инициализируем класс

In [431]:
mha = MultiHeadAttention(d_model=10, num_heads=2)

Сгенерируем батч из одного текста с двумя словами и размером эбмединга 3

In [432]:
inp = np.random.normal(size=(1, 2, 3)) 

In [433]:
inp

array([[[-1.15717704,  1.47875626, -0.21808616],
        [ 0.79722457, -0.69800946,  1.72175378]]])

Сформируем входной словарь для класса

In [434]:
inputs = {'query': inp,'key': inp, 'value': inp,}

Применим слой к входному словарю

In [121]:
result = mha(inputs)

На выходе получилось, что для каждого слова у нас теперь эмдединг размером 10 (мы указали такой d_model выше)

In [122]:
result.shape

torch.Size([1, 2, 10])

Давайте посмотрим как это вектор вычислялся внутри класса по шагам

In [123]:
# преобразуем эмбединг в q,k,v вектора
q = mha.query_dense(inputs['query'])
k = mha.query_dense(inputs['key'])
v = mha.query_dense(inputs['value'])

In [124]:
# размерность не изменилась так как мы еще не порезали на головы
q.shape

torch.Size([1, 2, 10])

In [126]:
# разрезаем каждый вектор на куски (головы)
qh = mha.split_heads(q, q.shape[0])
kh = mha.split_heads(k, k.shape[0])
vh = mha.split_heads(v, v.shape[0])

В результате получается тензор с размерностями (количество текстов x количество слов x количество голов x размерность вектора головы)

In [127]:
# мы указали num_heads 2 - поэтому размерность каждого куска 5 (10 разделить на два)
qh.shape

torch.Size([1, 2, 2, 5])

In [130]:
# keras.ops.transpose(kh).shape

Далее вычисляется скалярное произведение между q и k кусками. На вход подаются тензоры, но тензорфлоу умеет работать с таким форматом тоже

In [133]:
matmul_qk = keras.ops.matmul(qh, keras.ops.transpose(kh, axes=[0, 1, 3, 2]))

In [134]:
# На выходе получится тензор вот такой размерности
# 5 изменилось на 2
# так как для каждого куска у нас по две близости (до каждого слова включая себя)
matmul_qk.shape

torch.Size([1, 2, 2, 2])

Применим софтмакс, чтобы нагляднее увидеть близости

In [137]:
keras.ops.softmax(matmul_qk, axis=-1).detach().cpu().numpy().round(2)

array([[[[0.78, 0.22],
         [0.36, 0.64]],

        [[0.53, 0.47],
         [0.21, 0.79]]]], dtype=float32)

Далее эти близости используются как веса для суммы value кусков и потом результат по каждому кусочку собирается обратно в один вектор размерности (1, 2, 10)

### Маски

Еще один важный момент это то как используются маски, чтобы занулить некоторые близости.
Есть два вида масок - маска для паддинга и маска для слов в будущем. 
Маска для паддинга используется и в энкодере и в декодере, а маска для слов в будущем только для декодера.

In [138]:
def create_padding_mask(x):
    mask = keras.ops.cast(keras.ops.equal(x, PAD_IDX), torch.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, None, None, :]

In [139]:
create_padding_mask([[1,2,3]]) # 1 подставилась для индекса который равен PAD_IDX

tensor([[[[1., 0., 0.]]]], device='cuda:0')

Маска для будущего создает треугольную матрицу, которая накладывается на близости в self-attention части декодера. Как раз с помощью этой маски и реализуется разбиение входных последовательностей на контекст-продолжение - сначала вычисляется внимание до всех слов, даже в будущем, но потом полученные близости слов в будущем просто зануляются и не используются.

In [153]:
def create_look_ahead_mask(x):
    # эта функция немножко сложная, но суть у нее достаточно простая
    # нужно создать треугольную маску, с помощью которой мы закроем 
    # для каждого токена все последующие токены
    seq_len = x.shape[1]
    ones_mask = keras.ops.ones((1, seq_len, seq_len), dtype="int32")
    row_index = keras.ops.cumsum(ones_mask, axis=-2)
    col_index = keras.ops.cumsum(ones_mask, axis=-1)
    look_ahead_mask = ~ keras.ops.greater_equal(row_index, col_index)
    padding_mask = create_padding_mask(x)
    return keras.ops.maximum(look_ahead_mask, padding_mask)

In [160]:
# для декодера нам нужно замаскировать следующие токены
# так как мы пытаемся их сгенерировать
# для этого создается вот такая маска
ones_mask = keras.ops.ones((1, 3, 3), dtype="int32")
row_index = keras.ops.cumsum(ones_mask, axis=-2)
col_index = keras.ops.cumsum(ones_mask, axis=-1)
mask = ~ keras.ops.greater_equal(row_index, col_index)
mask

tensor([[[False,  True,  True],
         [False, False,  True],
         [False, False, False]]], device='cuda:0')

In [161]:
# в слое с вниманием эта маска будет использоваться чтобы занулить близости с токенами из будущего
(mask * -100)

tensor([[[   0, -100, -100],
         [   0,    0, -100],
         [   0,    0,    0]]], device='cuda:0')

In [162]:
# эта функция также проверяет и паддинг и если он есть то и его тоже замаскирует
# 1 - это паддинг айди
create_look_ahead_mask(torch.LongTensor([[1,3,2]]))

tensor([[[[1., 1., 1.],
          [1., 0., 1.],
          [1., 0., 0.]]]], device='cuda:0')

In [163]:
create_look_ahead_mask(torch.Tensor([[2,4,3]]))

tensor([[[[0., 1., 1.],
          [0., 0., 1.],
          [0., 0., 0.]]]], device='cuda:0')

In [164]:
create_look_ahead_mask(torch.Tensor([[2,4,3]]))

tensor([[[[0., 1., 1.],
          [0., 0., 1.],
          [0., 0., 0.]]]], device='cuda:0')

### Позиционное кодирование

К каждому эбмедингу слова добавляется вектор, который зависит от позиции слова. Таким образом в трансформер передается информация о порядке слов. Обратите внимание, что выше вектора сравниваются через скалярное произведение просто попарно и все вектора, которые не замаскированы, можно переставить местами и результат не изменится. Добавление информации о позиции в сами вектора изменяет результат.

Класс PositionalEncoding генерирует вектор для каждой позиции заданной размерности. Размерность задается также через d_model, чтобы можно было просто прибавить к вектору слова

In [258]:
class PositionalEncoding(keras.layers.Layer):

    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / keras.ops.power(10000, (2 * (i // 2)) / d_model)
        return keras.ops.multiply(position, angles)

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            position=torch.arange(0, position, dtype=torch.float32)[:, None],
            i=torch.arange(0, d_model, dtype=torch.float32)[None, :],
            d_model=d_model)
        sines = keras.ops.sin(angle_rads[:, 0::2])
        cosines = keras.ops.cos(angle_rads[:, 1::2])

        pos_encoding = keras.ops.concatenate([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[None, ...]
        return keras.ops.cast(pos_encoding, torch.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :inputs.shape[1], :]

### Encoder

Дальше кастомные функции и классы, которые были определены выше и стандартные инструменты tf.keras объединяются, чтобы собрать эндкодер и декодер. Энкодер устроен вот так (источник - https://jalammar.github.io/illustrated-transformer/)
![](https://jalammar.github.io/images/t/transformer_resideual_layer_norm.png)

Выше мы разобрались с self-attention и positional encoding. Теперь к этому нужно добавить еще один полносвязный слой (а точнее даже два), а также нормализацию и skip связи (пунктирные стрелочки). Ну и Dropout слои, но это просто случайное зануление некоторых векторов, чтобы модель лучше обучалась, а не что-то существенное, поэтому их даже нет на картинке.

Нормализация делается с помощью LayerNormalization слоя. Она нужна, чтобы значения получаемых векторов оставались в каких-то разумных пределах. А skip-связи реализованы просто через суммирование - обратите внимание, как переменная inputs сначала передается в MultiHeadAttention, а потом полученный результат прибавляется к исходной inputs; и далее attention (который является суммой inputs и MHA(inputs)) передается в два полносвязных слоя и полученный результат прибавляется к изначальной attention переменной. В итоге получается, что информация как бы постепенно добавляется к изначальным векторам, а не перезаписывается.

In [498]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
    inputs = keras.Input(shape=(None, d_model), name="inputs")
    padding_mask = keras.Input(shape=(1, 1, None), name="padding_mask")

    #call_mha
    attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
    attention = keras.layers.Dropout(rate=dropout)(attention)
    attention = keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

    outputs = keras.layers.Dense(units=units, activation='relu')(attention)
    outputs = keras.layers.Dense(units=d_model)(outputs)
    outputs = keras.layers.Dropout(rate=dropout)(outputs)
    outputs = keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

    return keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

Блоков энкодера может быть сколько угодно, так как один такой блок принимает на вход векторы размерности d_model и выдает как результат вектора размерности d_model, т.е. можно скармливать результаты одного блока другому и так далее. За количество последовательных транформерных блоков в энкодере отвечает параметр num_layers. Обратите внимание, что ниже все это реализовано просто через цикл с перезаписыванием переменной.

In [499]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name="encoder"):
    inputs = keras.Input(shape=(max_len,), name="inputs")
    padding_mask = keras.Input(shape=(1, 1, None), name="padding_mask")

    embeddings = keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= keras.ops.sqrt(keras.ops.cast(d_model, torch.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    #inputs (они тут называются outputs но это просто такой нейминг, 
    # этот параметр передается в encoder_layer первым и encoder_layer будет считать его inputs)
    # outputs он тут называется для удобства, так как он будет перезаписываться
    # чтобы на вход следующему блоку подавать уже не эмбединги,
    # а то что получится как результат предыдущего блока
    outputs = keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="encoder_layer_{}".format(i),
        )([outputs, padding_mask])

    return keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)

## Decoder

Декодер построен из точно таких же частей с одним важным отличием - в одном блоке декодера два MultiHeadAttention слоя. Первый из них такой же как в энкодере и реализует self-attention, а во втором реализован cross-attention. 

![](https://jalammar.github.io/images/t/transformer_resideual_layer_norm_3.png)

В коде разница выражает в параметрах, которые передаются в MultiHeadAttention. Обратите внимание, что ниже первый MultiHeadAttention вызывается также как и в энкодере (inputs={'query': inputs, 'key': inputs, 'value': inputs, 'mask': look_ahead_mask}), а вот на вход второму передаются inputs={ 'query': attention1, 'key': enc_outputs, 'value': enc_outputs, 'mask': padding_mask}, то есть query часть тут это inputs декодера уже обогащенный self-attention, а key и value это последние значения этих векторов в энкодере. Еще важно заметить, что в первом MultiHeadAttention используется look_ahead_mask, а во втором только padding_mask.

Таким образом, в декодере обращается внимание на уже сгенерированный перевод и на весь исходный текст. 

Обратите внимание, что вход декодера не может быть пустым, потому что тогда inputs будет пустым и не будет query, key, value векторов между которыми можно расчитать внимание и соответственно не из чего делать предсказания.

In [500]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = keras.Input(shape=(None, d_model), name="inputs")
    enc_outputs = keras.Input(shape=(None, d_model), name="encoder_outputs")
    look_ahead_mask = keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
    padding_mask = keras.Input(shape=(1, 1, None), name='padding_mask')

    attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })
    attention1 = keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

    attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })
    attention2 = keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

    outputs = keras.layers.Dense(units=units, activation='relu')(attention2)
    outputs = keras.layers.Dense(units=d_model)(outputs)
    outputs = keras.layers.Dropout(rate=dropout)(outputs)
    outputs = keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

    return keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [501]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name='decoder'):
    inputs = keras.Input(shape=(max_len,), name='inputs')
    enc_outputs = keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
    padding_mask = keras.Input(shape=(1, 1, None), name='padding_mask')

    embeddings = keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= keras.ops.sqrt(keras.ops.cast(d_model, torch.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    outputs = keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

### Модель целиком

Общая модель состоит из энкодера, декодера и еще 1 полносвязного слоя. Дополнительный последний слой нужен, чтобы из векторов получить вероятности классов (=слов)

In [502]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                max_len,
                name="transformer"):
    inputs = keras.Input(shape=(max_len[0],), name="inputs")
    dec_inputs = keras.Input(shape=(max_len[1]-1,), name="dec_inputs")

    enc_padding_mask = keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)
    
    look_ahead_mask = keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)
    
    dec_padding_mask = keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

    enc_outputs = encoder(
                          vocab_size=vocab_size[0],
                          num_layers=num_layers,
                          units=units,
                          d_model=d_model,
                          num_heads=num_heads,
                          dropout=dropout,
                          max_len=max_len[0],
                        )(inputs=[inputs, enc_padding_mask])

    dec_outputs = decoder(
                          vocab_size=vocab_size[1],
                          num_layers=num_layers,
                          units=units,
                          d_model=d_model,
                          num_heads=num_heads,
                          dropout=dropout,
                          max_len=max_len[1]-1,
                        )(inputs=[dec_inputs, enc_outputs, 
                                  look_ahead_mask, dec_padding_mask])

    outputs = keras.layers.Dense(units=vocab_size[1], name="outputs")(dec_outputs)

    return keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

Функция потерь тут реализована отдельно, чтобы не учитывать паддинги в расчетах. Поэтому выше последний полносвязный слой был без софтмакса - если применить софтмакс, а потом какие-то вероятности занулить, то распределение сломается (не будет суммироваться в 1). Поэтому указан параметр from_logits=True, который как раз означает, что на вход подаются не вероятности, а логиты, то есть просто результат полносвязного слоя без активации.


In [592]:

L  = keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none',)

def loss_function(y_true, y_pred):
    loss = L(y_true, y_pred)

    mask = keras.ops.cast(keras.ops.not_equal(y_true, PAD_IDX), torch.float32)
    loss = keras.ops.multiply(loss, mask)

    return keras.ops.mean(loss)

Определяем параметры модели. Есть стандартные наборы параметров для обучения маленькой, средней или большой модели. Их можно посмотреть в оригинальной статье про трансформер - https://arxiv.org/pdf/1706.03762.pdf

Мы возьмем маленький, т.к. он требует меньше ресурсов. НО для практической задачи есть смысл использовать модель побольше, есть статьи в которые показано, что большие трансформеры требуют даже меньше данных для обучения -  https://arxiv.org/pdf/2002.11794.pdf.

In [593]:
keras.backend.clear_session()

# small model
NUM_LAYERS = 2
D_MODEL = 256
NUM_HEADS = 8

UNITS = 512
DROPOUT = 0.1


# average model
# NUM_LAYERS = 6
# D_MODEL = 512
# NUM_HEADS = 8
# UNITS = 2048
# DROPOUT = 0.1


model = transformer(
    vocab_size=(tokenizer_en.get_vocab_size(),tokenizer_ru.get_vocab_size()),
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT,
    max_len=[max_len_en, max_len_ru])


optimizer = keras.optimizers.Adam(
    0.001, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
    return keras.metrics.sparse_categorical_accuracy(y_true, y_pred)


model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])
checkpoint = keras.callbacks.ModelCheckpoint('model_ruen.weights.h5',
                                            monitor='val_loss',
                                            verbose=1,
                                            save_weights_only=True,
                                            save_best_only=True,
                                            mode='min',
                                            save_freq='epoch')

Трансформеры - большие модели, обучаться они должны долго. Размер батча здесь уже имеет большое значение: сликом маленький батч приведет к сильному увеличению времени на обучение, но сильно большим его поставить не получится, т.к. модель большая и быстро займет всю видеокарту.

In [594]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [595]:
# model.load_weights('model_ruen.weights.h5')

In [596]:
model.fit((X_en_train, X_ru_dec_train), X_ru_out_train, 
             validation_data=((X_en_valid, X_ru_dec_valid), X_ru_out_valid),
             batch_size=200,
             epochs=3,
             callbacks=[checkpoint]
             )

Epoch 1/3
4750/4750 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.0894 - loss: 1.6173
Epoch 1: val_loss improved from inf to 0.99290, saving model to model_ruen.weights.h5
4750/4750 ━━━━━━━━━━━━━━━━━━━━ 708s 149ms/step - accuracy: 0.0894 - loss: 1.6173 - val_accuracy: 0.1475 - val_loss: 0.9929
Epoch 2/3
4750/4750 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.1476 - loss: 0.9682
Epoch 2: val_loss improved from 0.99290 to 0.89058, saving model to model_ruen.weights.h5
4750/4750 ━━━━━━━━━━━━━━━━━━━━ 708s 149ms/step - accuracy: 0.1476 - loss: 0.9682 - val_accuracy: 0.1599 - val_loss: 0.8906
Epoch 3/3
4750/4750 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - accuracy: 0.1591 - loss: 0.8709
Epoch 3: val_loss improved from 0.89058 to 0.85019, saving model to model_ruen.weights.h5
4750/4750 ━━━━━━━━━━━━━━━━━━━━ 708s 149ms/step - accuracy: 0.1591 - loss: 0.8709 - val_accuracy: 0.1654 - val_loss: 0.8502


Можно попробовать перевести рандомное предложение и оценить качество модели самому

In [597]:
@torch.no_grad()
def translate(text):
    input_ids = encode(text, tokenizer_en, target=False)

    input_ids = keras.ops.cast(keras.preprocessing.sequence.pad_sequences(
                                      [input_ids], maxlen=max_len_en, padding='post', 
                                       # важно не забыть паддинг с нужным id
                                       value=PAD_IDX), torch.int32)

    
    
    output_ids = [tokenizer_ru.token_to_id('[START]') ]
    
    pred = model((input_ids, keras.ops.cast(keras.preprocessing.sequence.pad_sequences(
                                      [output_ids], maxlen=max_len_ru-1, padding='post', 
                                       # важно не забыть паддинг с нужным id
                                       value=PAD_IDX), torch.int32))).cpu().numpy()
 
    
    while pred.argmax(2)[0][-1] not in [tokenizer_ru.token_to_id('[END]')]:
        if len(output_ids) >= max_len_ru:
            break
        # можно занизить скор тэга UNK чтобы он никогда не генерировался
        pred[:, :, tokenizer_ru.token_to_id('[UNK]')] = -100

        output_ids.append(pred.argmax(2)[0][-1])
        pred = model((input_ids, keras.ops.cast(keras.preprocessing.sequence.pad_sequences(
                                      [output_ids], maxlen=max_len_ru-1, padding='post', 
                                       # важно не забыть паддинг с нужным id
                                       value=PAD_IDX), torch.int32))).cpu().numpy()

    return tokenizer_ru.decode(output_ids[1:], )


In [600]:
translate("Transformer")

'трансформатортортортортор'

In [601]:
translate("can you translate this sentence?")

'можешь перевести это?'

In [602]:
translate("please translate this sentence into russian")

'пожалуйста, перевести это на это в русский'

In [395]:
# почему без старт и енд в енкодере хуже???
# ссылки на доп материалы
# как примменяется паддинг маск к атеншену

# BLEU

Оценивание машинного перевода очень сложная задача. Главная сложность в том, что хороших переводов одного текста может быть несколько, а есть обычно только 1. Технического решения этой проблемы пока не придумали и если нужна достоверная оценка, то используют людей или даже профессиональных переводчиков. Для автоматической оценки стандартно используется простая метрика BLEU. Она считает пересечения между двумя переводами на уровне отдельных токенов, биграммов, триграммов и четырехграммов. Чем больше полных пересечений, тем лучше. 

Реализация bleu есть в nltk

In [396]:
# %pip install nltk

In [603]:
import nltk

hypothesis = ['It', 'is', 'a', 'cat', 'at', 'room']
reference = ['It', 'is', 'a', 'cat', 'inside', 'the', 'room']


BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis, auto_reweigh=True)
print(BLEUscore)

0.4548019047027907


Давайте расчитаем BLEU на 10 примерах из тестового корпуса

In [609]:
text = open('opus.en-ru-test.ru').read().replace('\xa0', ' ')
f = open('opus.en-ru-test.ru', 'w')
f.write(text)
f.close()

In [610]:
en_sents_test = open('opus.en-ru-test.en').read().lower().splitlines()
ru_sents_test = open('opus.en-ru-test.ru').read().lower().splitlines()

In [611]:
translations = []

for i in range(10):
    translations.append(translate(en_sents_test[i]))

In [612]:
en_sents_test[:10]

['if you only stay there.',
 "i don't know how you do it, pop, carrying these boxes around every day.",
 'we might have a slight edge in mediation.',
 'how long is it going to take you to get him what he needs?',
 "on 1 april president of the nagorno karabagh republic bako sahakyan met head of the general staff of the republic of armenia's armed forces colonel-general yuri khachaturov.",
 'mr priesner also noted that the e-justice management system has not only improved case management, but has also led to a significant streamlining in procedures.',
 "you don't like chicken noodle soup?",
 'posted: 14 may 2005, 20:31',
 'now, for a minute, i thought maybe he was being tailed.',
 '« : 26 октябрь 2017, 06:50:24 »']

In [613]:
ru_sents_test[:10]

['только бы не вылететь.',
 'и как ты только справляешься, папа, таская эти коробки взад-вперед целый день.',
 'возможно, у нас есть небольшое преимущество в переговорах.',
 'сколько времени вы будете делать то, что ему нужно?',
 '1 апреля президент нкр бако саакян принял начальника генштаба вооруженных сил республики армения генерал-полковника юрия хачатурова.',
 'г-н приснер также упомянул, что система электронного правосудия не только позволила улучшить процесс ведения дел, но также способствует значительному упорядочению процедур.',
 '- неплохо, да.',
 'posted: 15 dec 2006, 00:07',
 'и на минутку я подумал, что за ним могут следить.',
 '«: 11 октябрь 2011, 17:15:34»']

In [614]:
translations

['если ты там не.',
 'я не знаю, как ты это делаешь,, все все...... я не знаю................ я не знаю.....',
 'у возможно небольшое в посредни дело. мы.,,, возможно,,,,,,,,, возможно,,,,,,,,,,,,,,,,,,.....',
 'сколько времени тебя нужно?',
 '1 президент нагорно карабахской республики бак бако саакянянянянянянянянянянянянянянянянааааааааяяяя..',
 'г - - - петербург г - - - - н шустернернерррр такжерр подчеркнулрррр,р подчеркнул,бкркррррр,.....',
 'тебе не нравится куриныйный суп??',
 'posted : 14 14 20 feb 2005,, 10 :',
 'я подумал,,, может он был,, он был на. и я думаю, он не не... я не не не не не не... не..........',
 '« ответ : # 30 10 10 2011 10, 11 10 11 11 13 14 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25 25']

BLEU очень сильно зависит от токенизации. В нашем случае это не проблема так как мы оцениваем одну модель. Но если бы мы хотели сравнить несколько, то нужно было бы создавать 1 общий токенизатор и рассчитывать через него

In [615]:
bleus = []

for i, t in enumerate(translations):
    reference = tokenizer_ru.encode(t).tokens
    hypothesis = tokenizer_ru.encode(ru_sents_test[i]).tokens

    bleus.append(round(nltk.translate.bleu_score.sentence_bleu([reference], hypothesis,  auto_reweigh=True), 3))

/home/ubuntu/.pyenv/versions/3.10.4/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/ubuntu/.pyenv/versions/3.10.4/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/ubuntu/.pyenv/versions/3.10.4/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Con

In [616]:
(sum(bleus)/len(bleus))*100

0.77

BLEU обычно показывают не от 0 до 1, а от 0 до 100. 

In [617]:
bleus

[0.0, 0.0, 0.0, 0.0, 0.077, 0.0, 0, 0.0, 0.0, 0.0]